In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [2]:
from client import Client
from server_algo import Server_with_Algorithm
from server import Server
from datasets import load_dataset, concatenate_datasets
import validation

In [3]:
# 1.首先选择要进行实验的client
clients = [Client(vectorstore_path="common_sense_db_0.6b"),
            Client(vectorstore_path="computer_science_coding_related_db_0.6b"),
            Client(vectorstore_path="law_related_db_0.6b"), Client(vectorstore_path="medicine_related_db_0.6b")]
# 2.对每个client进行加载
for c in clients:
    c.load_vectorstore()

/home/weiwei/RAGnet/client.py:63: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(model_name=model_path,


Vectorstore common_sense_db_0.6b loaded.
Vectorstore computer_science_coding_related_db_0.6b loaded.
Vectorstore law_related_db_0.6b loaded.
Vectorstore medicine_related_db_0.6b loaded.


In [4]:
# 3.创建server对象，这里需要选择模型，请预先在ollama上部署
# server = Server(model_name="qwen3:4b")
server = Server(model_name="qwen3:4b")

/home/weiwei/RAGnet/server.py:21: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  self.llm = ChatOllama(model=model_name)


In [5]:
# 4.加载测试数据集，这里以trivia_qa为例子，这里取前100个
from datasets import Dataset

samples = Dataset.from_parquet("./test_dataset/squad_full/plain_text/validation-00000-of-00001.parquet")

import random
random_samples = samples.shuffle(seed=42).select(range(100))

In [6]:
random_samples[0]['context']

'Private schooling in the United States has been debated by educators, lawmakers and parents, since the beginnings of compulsory education in Massachusetts in 1852. The Supreme Court precedent appears to favor educational choice, so long as states may set standards for educational accomplishment. Some of the most relevant Supreme Court case law on this is as follows: Runyon v. McCrary, 427 U.S. 160 (1976); Wisconsin v. Yoder, 406 U.S. 205 (1972); Pierce v. Society of Sisters, 268 U.S. 510 (1925); Meyer v. Nebraska, 262 U.S. 390 (1923).'

In [7]:
# 5.进行评估，并生成csv文件
validation.evaluate_datasets(clients=clients, server=server, samples=random_samples, top_k=20,
                              output_csv="squad_results_group1_topk_20.csv", dataset_name="squad")

Processing question: In what year did Massachusetts first require children to be educated in schools?
Question background: Read the following context carefully.         Do not explain your answer or include any additional text,             answer the question using **only** a span (exact phrase) from the context.
 Context: Private schooling in the United States has been debated by educators, lawmakers and parents, since the beginnings of compulsory education in Massachusetts in 1852. The Supreme Court precedent appears to favor educational choice, so long as states may set standards for educational accomplishment. Some of the most relevant Supreme Court case law on this is as follows: Runyon v. McCrary, 427 U.S. 160 (1976); Wisconsin v. Yoder, 406 U.S. 205 (1972); Pierce v. Society of Sisters, 268 U.S. 510 (1925); Meyer v. Nebraska, 262 U.S. 390 (1923).
Gold answers: ['1852', '1852', '1852']


uncleaned answer: <think>
Okay, I need to figure out what the user is asking here. They provided a bunch of context, mostly about various educational initiatives, technology programs, and some legal or policy stuff. The user hasn't actually asked a question yet, but maybe they want me to generate a question based on the given information.

Looking at the context, there's a lot about the Maine Learning Technology Initiative (MLTI), which provides laptops to students. There's also mention of other programs like MECC in Minnesota, RFID in schools, and some laws regarding biometrics in schools. There's also info on Massachusetts changing traffic laws, and some other educational institutions.

Since the user hasn't posed a specific question, maybe they want me to ask a question that's relevant to the provided context. The user might be testing if I can generate a question based on the given data. Alternatively, they might have intended to ask something but forgot. 

Looking at the history, 